In [1]:
import sys
sys.version

'2.7.15 |Anaconda, Inc.| (default, May  1 2018, 23:32:55) \n[GCC 7.2.0]'

In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
import cPickle as pickle
from core.solver import CaptioningSolver
from core.model import CaptionGenerator
from core.utils import load_coco_data
from core.bleu import evaluate

%matplotlib inline
plt.rcParams['figure.figsize'] = (8.0, 6.0)  # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

In [3]:
data = load_coco_data(data_path='data/', split='val')
with open('./data/train/word_to_idx.pkl', 'rb') as f:
    word_to_idx = pickle.load(f)

Loaded data/val/val.annotations.pkl..
features_path <type 'str'> 14
image_id <type 'numpy.ndarray'> (5000,) object
file_name <type 'numpy.ndarray'> (5000,) object
n_examples <type 'int'> 5000
tags <type 'numpy.ndarray'> (5000, 15) int32
Elapse time: 0.15


core/utils.py:30: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  data['image_id'] = annotations['image_id'].as_matrix()
core/utils.py:31: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  data['file_name'] = annotations['file_name'].as_matrix()


In [4]:
model = CaptionGenerator(word_to_idx, dim_feature=[196, 1024], dim_embed=512,
                                   dim_hidden=1536, n_time_step=31, prev2out=True, 
                                             ctx2out=True, alpha_c=1.0, selector=True, dropout=True)

In [5]:
solver = CaptioningSolver(model, data, data, n_epochs=20, batch_size=128, update_rule='adam',
                                      learning_rate=0.001, image_path='./image/val2017_resized',
                                pretrained_model=None, model_path='./model/lstm', test_model='./model_33000-33000',
                                 print_bleu=False, log_path='./log/')

In [6]:
solver.test(data, split='val', beam_size=3, attention_visualization=False)

INFO:tensorflow:Restoring parameters from ./model_33000-33000


100%|██████████| 40/40 [01:27<00:00,  2.19s/it]

Saved ./data/val/val.candidate.captions.json..


In [ ]:
test = load_coco_data(data_path='./data', split='test')

In [ ]:
tf.get_variable_scope().reuse_variables()
solver.test(test, split='test', beam_size=6)

In [ ]:
evaluate(data_path='./data', split='val')

In [ ]:
evaluate(data_path='./data', split='test')